In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
df = pd.read_csv("web_scraped_launch_data.csv",index_col=0)
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA (COTS)NRO,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA (COTS),Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA (CRS),Success,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA (CRS),Success,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10


In [2]:
df.isnull().sum()/df.count()*100

Flight No.         0.000000
Launch site        0.000000
Payload            0.000000
Payload mass       0.000000
Orbit              0.000000
Customer           0.000000
Launch outcome     0.000000
Version Booster    4.310345
Booster landing    0.000000
Date               0.000000
Time               0.000000
dtype: float64

In [3]:
df.dtypes

Flight No.          int64
Launch site        object
Payload            object
Payload mass       object
Orbit              object
Customer           object
Launch outcome     object
Version Booster    object
Booster landing    object
Date               object
Time               object
dtype: object

Looks like we need to change some of these to the correct type

In [4]:

df['Payload mass'] = df['Payload mass'].str.replace(' kg','')
df['Payload mass'] = df['Payload mass'].str.replace(',','')
# replace with specific number so we know to remove
# The C means it was classified
df['Payload mass'] = df['Payload mass'].str.replace('C','123456789')
df['Payload mass'] = df['Payload mass'].str.replace('~','')
df['Payload mass'] = df['Payload mass'].str.replace('5000–6000','6000')
df['Payload mass'] = df['Payload mass'].astype(int)
df['Booster landing'] = df['Booster landing'].str.replace('\n','')

In [5]:
df.dtypes

Flight No.          int64
Launch site        object
Payload            object
Payload mass        int32
Orbit              object
Customer           object
Launch outcome     object
Version Booster    object
Booster landing    object
Date               object
Time               object
dtype: object

In [6]:
df['Launch site'].value_counts()

CCAFS             40
KSC               33
Cape Canaveral    20
VAFB              16
CCSFS             12
Name: Launch site, dtype: int64

In [7]:
df.Orbit.value_counts()

LEO            67
GTO            33
Polar           7
SSO             7
MEO             3
HEO             2
Polar orbit     1
Sub-orbital     1
Name: Orbit, dtype: int64

remove failed launch

In [8]:
df.loc[df['Launch outcome']=='Failure']
df.drop([18],inplace=True)
df.reset_index()
df['Launch outcome'].value_counts()

Success    120
Name: Launch outcome, dtype: int64

In [9]:
df['Booster landing'].value_counts()

Success         80
No attempt      22
Failure         10
Controlled       5
Uncontrolled     2
Failure          1
Name: Booster landing, dtype: int64

Changing fails to 0 and success to 1

In [10]:
landing_outcomes =df['Booster landing'].value_counts()
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 Success
1 No attempt
2 Failure
3 Controlled
4 Uncontrolled
5 Failure 


In [11]:
bad_outcomes=set(landing_outcomes.keys()[[1,2,3,4,5]])
bad_outcomes

{'Controlled', 'Failure', 'Failure ', 'No attempt', 'Uncontrolled'}

In [12]:
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwise
landing_class=[]
for outcome in df['Booster landing']:
    if outcome in bad_outcomes:
        landing_class.append(0)
    else :
        landing_class.append(1)

In [13]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0


In [14]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time,Class
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.0B0003.1,Failure,4 June 2010,18:45,0
1,2,CCAFS,Dragon,0,LEO,NASA (COTS)NRO,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43,0
2,3,CCAFS,Dragon,525,LEO,NASA (COTS),Success,F9 v1.0B0005.1,No attempt,22 May 2012,07:44,0
3,4,CCAFS,SpaceX CRS-1,4700,LEO,NASA (CRS),Success,F9 v1.0B0006.1,No attempt,8 October 2012,00:35,0
4,5,CCAFS,SpaceX CRS-2,4877,LEO,NASA (CRS),Success,F9 v1.0B0007.1,No attempt,1 March 2013,15:10,0


In [15]:
df.drop('Launch outcome', axis=1, inplace=True)


In [16]:
df.drop('Payload', axis=1, inplace=True)


In [17]:
df.drop('Booster landing', axis=1, inplace=True)


In [18]:
df.to_csv('for_dash.csv')

In [4]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

#run jupyter viewer
# Read the airline data into pandas dataframe
max_payload = spacex_df['Payload mass'].max()
min_payload = spacex_df['Payload mass'].min()


pd.options.plotting.backend = "plotly"
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout

sites=spacex_df['Launch site'].unique().tolist()
sites.insert(0,'All Sites')




app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                   id='site-dropdown',
                                   options=[
                                    {'label': i, 'value': i} for i in sites],
                                    placeholder="Select a Launch Site here",
                                    value='All Sites',
                                    searchable=True),
                            
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0,
                                max=10000,
                                step=1000,
                                value=[min_payload,max_payload],
                                marks={
                                0: '0 kg',
                                2500: '2500',
                                5000: '5000',
                                7500: '7500',
                                10000: '10000'
                                }),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
Output(component_id='success-pie-chart', component_property='figure'),
Input(component_id='site-dropdown', component_property='value')
)
def get_pie(value):
    filtered_df = spacex_df
    if value == 'All Sites':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches By Site')
        return fig

    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == value].groupby(['Launch Site', 'class']). \
        size().reset_index(name='class count')
        title = f"Total Success Launches for site {value}"
        fig = px.pie(filtered_df,values='class count', names='class', title=title)
        return fig

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
Output(component_id='success-payload-scatter-chart', component_property='figure'),
[Input(component_id='site-dropdown', component_property='value'),
Input(component_id='payload-slider', component_property='value')]
)

def get_scatter(value1,value2):
    filtered_df2_1=spacex_df[(spacex_df['Payload Mass (kg)'] > value2[0]) & (spacex_df['Payload Mass (kg)'] < value2[1])]

    if value1=='All Sites':
        fig= px.scatter(filtered_df2_1,x="Payload Mass (kg)",y="class",color="Booster Version Category",\
        title="Correlation between Payload and Success for All sites")
        return fig
    else :
        filtered_df2_2=filtered_df2_1[filtered_df2_1['Launch Site']==value1]
        fig= px.scatter(filtered_df2_2,x="Payload Mass (kg)",y="class",color="Booster Version Category",\
        title=f"Correlation between Payload and Success for site {value1}")
        return fig
# Run the app
app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)


KeyError: 'Payload Mass (kg)'